In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import numpy as np
import pandas as pd
from train_utils import train_model, BCLogitMetric

In [2]:
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, recall_score

In [3]:
from classification_uncertainty_utils import aleatoric_uncertainty, epistemic_uncertainty, uncertainty_avg, score_avg

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
np.random.seed(1234)
torch.manual_seed(1234)

In [6]:
training_table = pd.read_table("./data/hill-valley/Hill_Valley_with_noise_Training.data", sep=',', dtype=np.float32)
testing_table = pd.read_table("./data/hill-valley/Hill_Valley_with_noise_Testing.data", sep=',', dtype=np.float32)

In [7]:
dim = 1
seq_length = 100

In [8]:
x_train = minmax_scale(training_table.drop("class", axis=1).values, axis=1)
y_train = training_table["class"].values
x_test = minmax_scale(testing_table.drop("class", axis=1).values, axis=1)
y_test = testing_table["class"].values

x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

N = len(y_train)

train_dl = torch.utils.data.DataLoader(
        [(x_train[i], y_train[i]) for i in range(len(y_train))],
        batch_size=10,
        shuffle=True
)
val_dl = torch.utils.data.DataLoader(
        [(x_val[i], y_val[i]) for i in range(len(y_val))],
        batch_size=10,
        shuffle=False
)
test_dl = torch.utils.data.DataLoader(
        [(x_test[i], y_test[i]) for i in range(len(y_test))],
        batch_size=len(y_test),
        shuffle=False
)

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = nn.LSTM(1, 10)
        self.fc = nn.Linear(10, 1)

    def regularizer(self):        
        return 0.

    def forward(self, x: torch.Tensor):
        out, _ = self.rnn(x)
        out = out[-1]
        out = torch.relu(out)
        out = self.fc(out).flatten()
        return out

In [10]:
net = Net()
criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [11]:
metrics = [BCLogitMetric()]

In [12]:
%%time
net = train_model(net, train_dl, val_dl, criterion, optimizer, "hill-valley.pt", 100, metrics=metrics, patience=5)

[1, 100.00%] train loss: 0.693061
[1, 100.00%] validate loss: 0.710546
Accuracy = 0.484
[2, 100.00%] train loss: 0.665555
[2, 100.00%] validate loss: 0.648465
Accuracy = 0.742
[3, 100.00%] train loss: 0.524788
[3, 100.00%] validate loss: 0.428899
Accuracy = 0.826
[4, 100.00%] train loss: 0.350633
[4, 100.00%] validate loss: 0.298503
Accuracy = 0.868
[5, 100.00%] train loss: 0.249484
[5, 100.00%] validate loss: 0.222077
Accuracy = 0.893
[6, 100.00%] train loss: 0.188720
[6, 100.00%] validate loss: 0.162065
Accuracy = 0.911
[7, 100.00%] train loss: 0.152449
[7, 100.00%] validate loss: 0.127921
Accuracy = 0.924
[8, 100.00%] train loss: 0.127388
[8, 100.00%] validate loss: 0.105772
Accuracy = 0.933
[9, 100.00%] train loss: 0.110659
[9, 100.00%] validate loss: 0.090516
Accuracy = 0.941
[10, 100.00%] train loss: 0.099011
[10, 100.00%] validate loss: 0.078570
Accuracy = 0.947
[11, 100.00%] train loss: 0.089809
[11, 100.00%] validate loss: 0.069626
Accuracy = 0.952
[12, 100.00%] train loss: 0.

### Test

In [13]:
net.eval()
with torch.no_grad():
    for i, (x, y) in enumerate(test_dl):
        pre_sigmoids = net(x.transpose(-2,-3))
        scores = torch.sigmoid(pre_sigmoids)
        predictions = torch.round(scores)
        print("Accuracy = %.2f" % accuracy_score(y, predictions))
        print("Precision = %.2f" % precision_score(y, predictions))
        print("AUCROC = %.2f" % roc_auc_score(y, scores))

Accuracy = 1.00
Precision = 1.00
AUCROC = 1.00


### Flat line test

In [ ]:
new_x_test = minmax_scale(np.full((2, 100), 0., dtype=np.float32), axis=1)
new_x_test = np.expand_dims(new_x_test, axis=-1)
new_x_test = new_x_test.swapaxes(-2,-3)
new_x_test = torch.tensor(new_x_test)

net.eval()
with torch.no_grad():
    outputs = net(new_x_test)
    outputs = torch.sigmoid(outputs)
    scores = outputs

plt.plot(new_x_test[:,0])
plt.show()
print("predictions", scores)

### Noise test

In [ ]:
new_x_test = minmax_scale(np.random.randn(2, 100).astype(np.float32), axis=1)
new_x_test = np.expand_dims(new_x_test, axis=-1)
new_x_test = new_x_test.swapaxes(-2,-3)
new_x_test = torch.tensor(new_x_test)

net.eval()
with torch.no_grad():
    outputs = net(new_x_test)
    outputs = torch.sigmoid(outputs)
    scores = outputs

plt.plot(new_x_test[:,0])
plt.show()
print("predictions", scores)